In [69]:
%rebootdevice

normal repl mode not detected b'\r\nMicroPython 0fb6bf7 on 2021-05-08; Raspberry Pi with ARM1176JZF-S\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\n\r\nmounting SD card...'
not command mode
done
MicroPython 0fb6bf7 on 2021-05-08; Raspberry Pi with ARM1176JZF-S
Type "help()" for more information.
>>> 
>>> 
>>> 
>>> 
raw REPL; CTRL-B to exit
>

In [70]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [63]:
%sendtofile --source "../Sensor_Kennel/BME280_funcs.py"
%sendtofile --source "st7735_funcs.py"


Sent 70 lines (2402 bytes) to st7735_funcs.py.


In [57]:
%serialconnect


serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [72]:
%sendtofile main.py

from machine import I2C, Pin
import time, os, sys, ustruct
import BME280_funcs
from st7735_funcs import fbuf, display

print("platform", sys.platform)
if sys.platform == "esp32":
    i2c = I2C(1, scl=Pin(25), sda=Pin(27)) # on ESP32 pin26 broken
    from machine import UART
    u = UART(1, tx=5, rx=4, baudrate=9600)
    print(u)
else:
    i2c = I2C(1)  # on Pi
    u = None
    gasheaterenable = Pin(24, Pin.OUT)
    gasheaterenable.value(1)
print("i2c devices", list(map(hex, i2c.scan())))  # 0x23=LTR-559, 0x76=BME280, 0x49=ADS1015
time.sleep_ms(1000)

# initialize all the sensors
BME280_funcs.bme280init(i2c, 0x76)
print("LTR-559 part_number/revision", i2c.readfrom_mem(0x23, 0x86, 1), "should be 9 2")
i2c.writeto_mem(0x23, 0x80, b'\x09')  # enable ALS gain x4 (light sensor)

dustcols = ["apm10", "apm25", "apm100", "pm10", "pm25", "pm100",
            "gt03um", "gt05um", "gt10um", "gt25um", "gt50um", "gt100um" ]
def readdust():
    if u == None:
        return None
    x = u.read()
    if x and len(x) == 32 and x[0] == 0x42 and x[1] == 0x4d:
        numbers = ustruct.unpack('>16H', x)
        csum = sum(x[:-2])
        if csum == numbers[-1]:
            #assert(numbers[1] == 28) # frame length
            return dict(zip(dustcols, numbers[2:-2]))
        else:
            print("bad checksum", x)
    return None


# Numbers all seem wrong coming out here (should attach 3.3V to the heater enable pin)
# start conversion | channel | gain(FSR) | single shot | 1600 SPS | disable comparator
channels = [0x4000, 0x5000, 0x6000, 0x7000]  # in0/gnd, in1, in2, in3 (fixed at 1.25V reference)
gainfsr, gainV = 0x0000, 6.144
def convertgasses():
    vals = [ ]
    for channel in channels:
        i2c.writeto_mem(0x49, 0x01, ustruct.pack(">H", 0x8000 | channel | gainfsr | 0x0100 | 0x0080 | 0x0003))
        time.sleep_ms(10)
        for j in range(10000):
            x = i2c.readfrom_mem(0x49, 0x01, 2)
            x1 = ustruct.unpack(">H", x)[0]
            if (x1 & 0x8000):  # conversion complete
                break
        r = ustruct.unpack(">h", i2c.readfrom_mem(0x49, 0x00, 2))
        #vals.append(r[0]>>3)
        v = (r[0] >> 3)/2048.0*gainV
        vals.append(v)
    #print("ADS1015", "ox", 56000*(3.3-vals[0]), "red", 56000*(3.3-vals[1]), "nh3", 56000*(3.3-vals[2]), "ref", vals[3])
    return 56000*(3.3-vals[0]), 56000*(3.3-vals[1]), 56000*(3.3-vals[2]), vals[3]

while True:
    time.sleep_ms(100)
    try:
        temp, baro, humid = BME280_funcs.readBME280(0x76)
        fbuf.fill_rect(0, 0, 80, 32, 0x0000)
        fbuf.text("  BME280", 0, 0, 0xFFFF)
        fbuf.text("tmp %f"%temp, 0, 8, 0xFFFF)
        fbuf.text("prs %f"%baro, 0, 16, 0xFFFF)
        fbuf.text("h%% %f"%humid, 0, 24, 0xFFFF)
    except OSError as e:
        print("OSError", e)

    try:
        ltrval = ustruct.unpack("h", i2c.readfrom_mem(0x23, 0x8A, 2))[0]
        fbuf.fill_rect(0, 32, 80, 8, 0x0000)
        fbuf.text("LTR %d"%ltrval, 0, 32, 0xFFFF)
    except OSError as e:
        print("OSError", e)

    try:
        gasox, gasred, gasnh3, gasref = convertgasses()
        fbuf.fill_rect(0, 40, 80, 40, 0x0000)
        fbuf.text("  Gasses", 0, 40, 0xFFFF)
        fbuf.text("ox %f" % gasox, 0, 48, 0xFFFF)
        fbuf.text("red %f" % gasred, 0, 56, 0xFFFF)
        fbuf.text("nh3 %f" % gasnh3, 0, 64, 0xFFFF)
        fbuf.text("ref %f" % gasox, 0, 72, 0xFFFF)
    except OSError as e:
        print("OSError", e)

    d = readdust()
    if d:
        fbuf.fill_rect(0, 80, 80, 56, 0x0000)
        fbuf.text("  Dust", 0, 80, 0xFFFF)
        fbuf.text("03u %d" % d["gt03um"], 0, 88, 0xFFFF)
        fbuf.text("05u %d" % d["gt05um"], 0, 96, 0xFFFF)
        fbuf.text("10u %d" % d["gt10um"], 0, 104, 0xFFFF)
        fbuf.text("25u %d" % d["gt25um"], 0, 112, 0xFFFF)
        fbuf.text("50u %d" % d["gt25um"], 0, 120, 0xFFFF)
        fbuf.text("100u %d" % d["gt100um"], 0, 128, 0xFFFF)
        
    display()
       

Sent 105 lines (3927 bytes) to main.py.
